In [4]:
import pandas as pd
import yfinance as yf
from fredapi import Fred

In [7]:
fred = Fred(api_key="d6e700bbd631aa30c4c24bcb5f3a8150")

fred_series = {
    "Interest_Rate": "FEDFUNDS",        # Fed Funds Rate
    "10Y_Treasury_Yield": "DGS10",      # 10-Year Treasury Yield
    "Inflation_CPI": "CPIAUCSL",        # Consumer Price Index
    "Unemployment": "UNRATE"            # Unemployment Rate
}


In [19]:
def get_stock_data(ticker, start="2020-01-01", end="2025-01-01"):
    stock = yf.Ticker(ticker)

    #OHLCV price data
    df = stock.history(start = start , end = end)
    df = df[["Open", "High", "Low", "Close", "Volume"]]

    #SMA & EMA
    df["SMA_20"] = df["Close"].rolling(window=20).mean()
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()

    #RSI
    delta = df["Close"].diff()
    gain = (delta.where(delta >0,0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100/(1+rs))

    #MACD & Signal
    ema_12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema_26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema_12 - ema_26
    df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

    #Volatility
    df["Volatility_20"] = df["Close"].rolling(window=20).std()

    return df

In [22]:
def get_fred_data(series_dict, start = "2020-01-01", end="2025-01-01"):
    data = {}
    for name, code in series_dict.items():
        s = fred.get_series(code, start, end)
        data[name] = s
    df = pd.DataFrame(data)
    return df

In [ ]:
apple_df = get_stock_data("AAPL", start="2020-01-01", end="2025-01-01")

macro_df = get_fred_data(fred_series, start="2020-01-01", end="2025-01-01")

# Make both indices timezone-naive
apple_df.index = apple_df.index.tz_localize(None)
macro_df.index = macro_df.index.tz_localize(None)

merged_df = apple_df.merge(macro_df, left_index=True, right_index=True, how="outer")
merged_df = merged_df.sort_index().fillna(method="ffill")

merged_df.to_csv("APPL_with_macro_2020_2025.csv")

print("Combined dataset saved as 'AAPL_with_macro_2020_2025'")
print(merged_df.head())

Combined dataset saved as 'AAPL_with_macro_2020_2025'
                 Open       High        Low      Close       Volume  SMA_20  \
2020-01-01        NaN        NaN        NaN        NaN          NaN     NaN   
2020-01-02  71.545890  72.598892  71.292304  72.538513  135480400.0     NaN   
2020-01-03  71.765651  72.594040  71.608669  71.833275  146322800.0     NaN   
2020-01-06  70.954181  72.444313  70.703005  72.405670  118387200.0     NaN   
2020-01-07  72.415345  72.671348  71.845377  72.065155  108872000.0     NaN   

               EMA_20  RSI_14      MACD    Signal  Volatility_20  \
2020-01-01        NaN     NaN       NaN       NaN            NaN   
2020-01-02  72.538513     NaN  0.000000  0.000000            NaN   
2020-01-03  72.471348     NaN -0.056258 -0.011252            NaN   
2020-01-06  72.465093     NaN -0.054033 -0.019808            NaN   
2020-01-07  72.427003     NaN -0.078837 -0.031614            NaN   

            Interest_Rate  10Y_Treasury_Yield  Inflation_CPI  

C:\Users\Nam Nguyen\AppData\Local\Temp\ipykernel_40984\1846812741.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df = merged_df.sort_index().fillna(method="ffill")


In [35]:
# Load with Date as index
merged_df = pd.read_csv(
    r"E:\Study\usth\fundamental_data_science\crawl_data\APPL_with_macro_2020_2025.csv",
    index_col=0,      # use first column as index
    parse_dates=True  # parse as datetime
)

# (Optional) reset index if you want Date as a column instead of index
# merged_df = merged_df.reset_index().rename(columns={'index': 'Date'})

# Save again without extra number column
merged_df.to_csv("APPL_with_macro_2020_2025.csv", index=False)

C:\Users\Nam Nguyen\AppData\Local\Temp\ipykernel_40984\1163949872.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df = pd.read_csv(


In [36]:
print(merged_df.head())

         Date       Open       High        Low      Close       Volume  \
0  2020-01-01        NaN        NaN        NaN        NaN          NaN   
1  2020-01-02  71.545890  72.598892  71.292304  72.538513  135480400.0   
2  2020-01-03  71.765651  72.594040  71.608669  71.833275  146322800.0   
3  2020-01-06  70.954181  72.444313  70.703005  72.405670  118387200.0   
4  2020-01-07  72.415345  72.671348  71.845377  72.065155  108872000.0   

   SMA_20     EMA_20  RSI_14      MACD    Signal  Volatility_20  \
0     NaN        NaN     NaN       NaN       NaN            NaN   
1     NaN  72.538513     NaN  0.000000  0.000000            NaN   
2     NaN  72.471348     NaN -0.056258 -0.011252            NaN   
3     NaN  72.465093     NaN -0.054033 -0.019808            NaN   
4     NaN  72.427003     NaN -0.078837 -0.031614            NaN   

   Interest_Rate  10Y_Treasury_Yield  Inflation_CPI  Unemployment  
0           1.55                 NaN        259.127           3.6  
1           1.55